In [3]:
import os
import requests


# Set your Confluence URL, email, and API token
confluence_url = "https://mikesofts.atlassian.net/"
email = os.environ['EMAIL']
api_token = os.environ['API_TOKEN']

# Define your Confluence API URL
base_url = "https://mikesofts.atlassian.net/wiki/rest/api"

# Create a session for authentication
session = requests.Session()
session.auth = (email, api_token)

# Get the list of all pages
def get_all_pages():
    pages = []
    start = 0
    limit = 50  # You can adjust the limit based on your needs
    while True:
        url = f"{base_url}/content"
        params = {
            "start": start,
            "limit": limit,
            "expand": "version,body.view",
        }
        response = session.get(url, params=params)

        if response.status_code != 200:
            print(f"Error: {response.status_code}")
            break

        data = response.json()
        pages.extend(data.get("results", []))

        if data.get("size", 0) < limit:
            break

        start += limit

    return pages

# Example usage
if __name__ == "__main__":
    all_pages = get_all_pages()
    print(f"Total pages: {len(all_pages)}")

    for page in all_pages:
        print(f"Title: {page['title']}, ID: {page['id']}")



Total pages: 26
Title: mamba setup, ID: 33113
Title: Knowledge base, ID: 65638
Title: Template - How-to guide, ID: 65650
Title: Template - Troubleshooting article, ID: 65664
Title: Getting started in Confluence, ID: 98394
Title: Overview, ID: 98395
Title: Overview, ID: 98634
Title: Software Development, ID: 262245
Title: Template - Product requirements, ID: 262284
Title: Template - Meeting notes, ID: 262298
Title: Template - Decision documentation, ID: 262312
Title: Get the most out of your software project space, ID: 262326
Title: baby mamba, ID: 262359
Title: Data Science, ID: 884737
Title: Artificial intelligence, ID: 884784
Title: In fiction, ID: 885246
Title: Etymology, ID: 917667
Title: Ethics, ID: 918001
Title: Philosophy, ID: 918153
Title: Future, ID: 918165
Title: Foundations, ID: 983041
Title: Data Science and Data Analysis, ID: 983053
Title: Goals, ID: 1048724
Title: Tools, ID: 1048904
Title: Applications, ID: 1048916
Title: History, ID: 1049028
